In [4]:
!pip install -r ../requirements.txt

  Obtaining dependency information for asttokens==3.0.0 from https://files.pythonhosted.org/packages/25/8a/c46dcc25341b5bce5472c718902eb3d38600a903b14fa6aeecef3f21a46f/asttokens-3.0.0-py3-none-any.whl.metadata
  Obtaining dependency information for certifi==2024.8.30 from https://files.pythonhosted.org/packages/12/90/3c9ff0512038035f59d279fddeb79f5f1eccd8859f06d6163c58798b9487/certifi-2024.8.30-py3-none-any.whl.metadata
  Obtaining dependency information for charset-normalizer==3.4.0 from https://files.pythonhosted.org/packages/0b/6e/b13bd47fa9023b3699e94abf565b5a2f0b0be6e9ddac9812182596ee62e4/charset_normalizer-3.4.0-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for click==8.1.8 from https://files.pythonhosted.org/packages/7e/d4/7ebdbd03970677812aac39c869717059dbb71a4cfc033ca6e5221787892c/click-8.1.8-py3-none-any.whl.metadata
  Obtaining dependency information for comm==0.2.2 from https://files.pythonhosted.org/packages/e6/75/49e5bfe642f71f272236b5b2d2691cf915a

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gensim 4.3.0 requires FuzzyTM>=0.4.0, which is not installed.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.1.3 which is incompatible.


In [1]:
# importing necessary dependecies
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np


ModuleNotFoundError: No module named 'seaborn'

In [87]:

train_file = "../data/raw/train.csv"
df = pd.read_csv(train_file)

In [ ]:
df.head()

In [ ]:
df['number_vmail_messages'].unique()

In [ ]:
df.columns

In [ ]:
df.info()

In [ ]:
df.dtypes

In [ ]:
df.duplicated().sum()

In [ ]:
cat_columns = [feature for feature in df.columns if feature in df.select_dtypes(['object']).columns]

cat_columns

In [ ]:
# col_count = []
# for cat in cat_columns:
#     count = df[cat].value_counts().shape[0]
#     col_count.append({'column' : cat, 'count' : count})

test_count = [{'column' : cat, 'count' : df[cat].nunique()} for cat in cat_columns]
cat_df = pd.DataFrame(test_count)

cat_df

In [ ]:
for i in range(1, len(cat_columns)):
    plt.figure(figsize=(4,2))
    sns.countplot(x=cat_columns[i], data=df)
    plt.show()
                  

In [ ]:
# selecting numerical variables

df_num = df.select_dtypes(['int', 'float'])

df_num.head()

In [ ]:
sns.set_style('dark')
df.hist(bins=50, figsize=(20,20), color='navy')
plt.show()

In [ ]:
# To visualize the outliers using box plots

for col in df_num.columns:
    plt.figure(figsize=(3,4))
    # sns.boxplot(data=df, x=df['churn'], y=col)
    sns.boxplot(data=df_num, y=col)
    plt.title('Visualize Outliers')
    plt.tight_layout()
    plt.show()

In [ ]:
# creating a function to count outliers 

def outlier_table_count(df, columns, handle_outliers=False):
    outlier_counts = []
    
    for col in columns:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        
        outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
        outlier_count = outliers.shape[0]
        
        outlier_counts.append({'column' : col, 'outlier' : outlier_count})
        
        if handle_outliers:
            # using .clip()
            # df[col] = df[col].clip(lower = lower_bound, upper = upper_bound)
            
            df[col] = np.where(df[col] < lower_bound, lower_bound, df[col])
            df[col] = np.where(df[col] > upper_bound, upper_bound, df[col])
            
    outlier_df = pd.DataFrame(outlier_counts)
    
    return outlier_df


outlier_table_count(df_num, df_num.columns, handle_outliers=True)
        
        
        


In [ ]:
df_num

In [ ]:
df_num.hist(bins=50, figsize=(10,10), color='navy')
plt.show()